## CNN Model V2

In [309]:
# Load packages
import csv
import numpy as np
import os
import random

from PIL import Image

### Variables and Hyperparameters

In [348]:
###############
## VARIABLES ##
###############

# name of directory with fire tif files
directory = "toydata"

#####################
## HYPERPARAMETERS ##
#####################

# the desired height and width (in pixels) of the matrix to feed into the CNN
# 1 pixel side = 500 meters = 0.310686 miles
matrix_dim = 32

# multiplier for amount of zero-labeled data we want to add to dataset
labeled_multiplier = 1

# test size for train/test split
test_size = 0.2

# training epochs
epoc = 10

### Dataset Preprocessing Pipeline

In [349]:
def data_processing(directory):
    '''
    Process the dataset in the supplied directory and return matrices of which pixels belong to which fire and 
    which day of the year the pixel was on fire.
    
    Args: 
        - directory: name of directory with tif files
    Returns: 
        - fire_data_dict: a dictionary where the key is "fire_id" and the value is a matrix of pixels 
        triggered by that fire (0, 1)
        - fireline: matrix denoting what day of year that pixel was on fire (1-365)
    '''
    
    
    path = os.path.abspath(directory)

    tiff_files = []

    for f in os.listdir(path):
        if f.endswith('.tif'):
            tiff_files.append(path + '/' + f)

    tiff_dict = {}

    # dictionary of tiff files
    for f in tiff_files:
        k = f.split('/')[-1].split('.tif')[0]
        tiff_dict[k] = f

    # convert to np array
    fire_id = Image.open(tiff_dict['fireid'])
    fire_id = np.array(fire_id)
    fire_id[fire_id == -9999] = 0

    fireline = Image.open(tiff_dict['Global_fire_atlas_firelinecrop'])
    fireline = np.array(fireline)
    fireline[fireline == -9999] = 0

    # get list of unique fire_ids
    fire_ids = set()

    for row in fire_id:
        for val in row:
            fire_ids.add(val)

    # remove 0 from fire_ids set because it does not denote a fire
    fire_ids.remove(0)

    # get dict with key value pairs of fire_id and an empty dict
    fire_data_dict = {}

    for id in fire_ids:
        id = str(id)
        fire_data_dict[id] = {}

    for id in fire_ids:
        indices = np.where(fire_id == id, 1, 0)
        fire_data_dict[str(id)] = indices
        
    return fire_data_dict, fireline

In [350]:
def create_one_hot_matrices(data_dict, fireline):
    '''
    Create matrices for each fire_id that show were the fire was on a given day during the year.
    
    Args:
        - data_dict: a dictionary where the key is "fire_id" and the value is a matrix of pixels 
        triggered by that fire (0, 1)
        - fireline: matrix denoting what day of year that pixel was on fire (1-365)
    Returns:
        - fire_data_dict: a dictionary of the following structure:
            {
                "fire_id": {
                    "day_of_year": one-hot encoded 2D array of fire spread on that day,
                    "day_of_year": one-hot encoded 2D array of fire spread on that day
                }

            }
    '''
    
    fire_data_dict = {}

    for key, val in data_dict.items():
        data = {}
                
        for y in range(1, 366):
            mask = ((fireline == y) & (val == 1))
            mask = mask.astype(int)
        
            if np.sum(mask) > 0:
                data[str(y)] = mask
        
        fire_data_dict[key] = data
        
    return fire_data_dict

In [351]:
def create_day_pairs(fire_data_dict):
    '''
    Create a list of sets where the first value is where the fire was on a given day and the second value is where
    the fire was on the following day.
    
    Args:
        - fire_data_dict: a dictionary of the following structure:
            {
                "fire_id": {
                    "day_of_year": one-hot encoded 2D array of fire spread on that day,
                    "day_of_year": one-hot encoded 2D array of fire spread on that day
                }

            }
    Returns:
        - train_labels: a list of sets where the first value of the set is a one-hot encoded 2D array of fire 
        spread on day_1 and the second value of the set is a one-hot encoded 2D array of fire spread on day_2:
        [
            (one-hot encoded 2D array of fire spread on that day_1, one-hot encoded 2D array of fire spread on day_2),
            (one-hot encoded 2D array of fire spread on that day_2, one-hot encoded 2D array of fire spread on day_3),
        ]
    '''
    
    train_labels = []

    for key, value in fire_data_dict.items():
        burn_matrices = list(value.values())
        
        for index, day in enumerate(burn_matrices):

            if index < len(burn_matrices) - 1:
                day_1 = burn_matrices[index]
                day_2_index = index + 1
                day_2 = burn_matrices[day_2_index]
                
                pair = (day_1, day_2)
                train_labels.append(pair)

    return train_labels

In [352]:
def balance_dataset(dataset, matrix_dim, num_pixels, side):
    '''
    Supplement the list produced in `create_labeled_data` with data where there was no data
    
    Args:
        - dataset: a list of sets where the first value of the set is a one-hot encoded 2D array of fire spread 
        on day_1 and the second value of the set is a one-hot encoded 2D array of fire spread on day_2
        - matrix_dim: a hyperparameter for the height and width of the matrices fed into the CNN
        - num_pixels: how many "no-fire" pixel-matrix pairs we want to return
        - side: half the length of the dimension of the outpur matrix
    Returns:
        - no_fire: a list of sets, where the second value (0, 1) represents whether fire is present for a given pixel, and the
        first value is a matrix centered on the second value for the previous day and represents where the fire was
        on the previous day
    '''
        
    no_fire = []

    for (x, y) in dataset:    

        x = np.pad(x, pad_width=matrix_dim, mode='constant', constant_values=0)
        y = np.pad(y, pad_width=matrix_dim, mode='constant', constant_values=0)

        vals = np.where(y == 0)
        vals = list(zip(vals[0], vals[1]))

        for (xi, yi) in vals:
            xi_r = xi + side
            xi_l = xi - side
            yi_b = yi + side
            yi_t = yi - side

            m = x[xi_l:xi_r, yi_t:yi_b]
            
            # control for edge cases where shape doesn't match up - not sure why this is happening
            if m.shape == (matrix_dim, matrix_dim):
                no_fire.append((m, 0))
    
    no_fire = random.sample(no_fire, num_pixels)
    
    return no_fire

In [353]:
def create_labeled_data(dataset, matrix_dim):
    '''
    Create a list of sets where the first value is a matrix of pixels on a given day and the second value denotes
    whether there was fire in the center pixel on the following day.
    
    Args:
        - dataset: a list of sets where the first value of the set is a one-hot encoded 2D array of fire spread 
        on day_1 and the second value of the set is a one-hot encoded 2D array of fire spread on day_2
        - matrix_dim: a hyperparameter for the height and width of the matrices fed into the CNN
    Returns:
        - data: a list of sets, where the second value (0, 1) represents whether fire is present for a given pixel, and the
        first value is a matrix centered on the second value for the previous day and represents where the fire was
        on the previous day
    '''

    side = int(matrix_dim/2)
    
    data = []
    
    for (x, y) in dataset:    

        x = np.pad(x, pad_width=matrix_dim, mode='constant', constant_values=0)
        y = np.pad(y, pad_width=matrix_dim, mode='constant', constant_values=0)

        vals = np.where(y == 1)
        vals = list(zip(vals[0], vals[1]))

        for (xi, yi) in vals:
            xi_r = xi + side
            xi_l = xi - side
            yi_b = yi + side
            yi_t = yi - side

            m = x[xi_l:xi_r, yi_t:yi_b]

            data.append((m, 1))
    
    data_len = len(data)
    num_pixels = min(int(data_len*labeled_multiplier), data_len)
    
    # balance this dataset with values where there is no fire
    no_fire = balance_dataset(dataset, matrix_dim, num_pixels, side)
    
    # combine and shuffle
    data += no_fire    
    random.shuffle(data)
    
    return data

In [354]:
def prep_dataset_for_cnn(data):
    '''
    Takes a list of (matrix, integer) pairs and returns input data and output labels split into train and test sets
    
    Args:
        - data: a list of (matrix, integer) pairs
    Returns:
        - X: array of input data in matrix_dim X matrix_dim shape
        - Y: array of output labels (0 or 1)
    '''
    
    X = []
    Y = []

    for (x, y) in data:
        x = np.asarray(x)
        X.append(x)
        
        Y.append(y)

    X = np.asarray(X)
    Y = np.asarray(Y)
    
    obs = len(X)
    
    X = X.reshape(obs, matrix_dim, matrix_dim, 1)

    return X, Y

### Run Data Preprocessing Pipeline

In [355]:
fire_data_dict, fireline = data_processing(directory)
fire_data_dict = create_one_hot_matrices(fire_data_dict, fireline)
small_dataset = create_day_pairs(fire_data_dict)
data = create_labeled_data(small_dataset, matrix_dim)
X, Y = prep_dataset_for_cnn(data)

## Build CNN

In [356]:
# import packages

from __future__ import print_function

import tensorflow as tf

import keras
import keras.backend as K

from keras.models import Sequential, Model
from keras.layers import AveragePooling2D, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input

In [357]:
# compute f1 score manually

def recall_m(y_true, y_pred):
    true_pos = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_pos = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_pos / (possible_pos + K.epsilon())

    return recall

def precision_m(y_true, y_pred):
    true_pos = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_pos = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_pos / (predicted_pos + K.epsilon())
    
    return precision

def f1_score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    score = 2*((precision*recall)/(precision+recall+K.epsilon()))
    
    return score

### Model 1 - Fire Image Data

In [358]:
# Create model_1: fire image data with Sequential API
model_1 = Sequential()

# Add layers
model_1.add(AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid'))
model_1.add(Conv2D(64, kernel_size=(3, 3), activation='sigmoid'))
model_1.add(Conv2D(32, kernel_size=(3, 3), activation='sigmoid'))
model_1.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid'))
model_1.add(Dropout(0.2))
model_1.add(Flatten())

# Final dense layer 
model_1.add(Dense(1, activation='sigmoid'))

In [359]:
# compile the model
model_1.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy', f1_score, tf.keras.metrics.AUC()]
)

In [360]:
# fit the model
model_1.fit(
    x = X, 
    y = Y,
    validation_split = test_size, 
    epochs=epoc
)

Train on 5409 samples, validate on 1353 samples
Epoch 1/10
5409/5409 [==============================] - 2s 375us/step - loss: 0.6989 - accuracy: 0.5271 - f1_score: 0.4644 - auc_49: 0.5153 - val_loss: 0.6807 - val_accuracy: 0.8684 - val_f1_score: 0.8378 - val_auc_49: 0.5400
Epoch 2/10
5409/5409 [==============================] - 2s 329us/step - loss: 0.5743 - accuracy: 0.6900 - f1_score: 0.6678 - auc_49: 0.5972 - val_loss: 0.3772 - val_accuracy: 0.9157 - val_f1_score: 0.9014 - val_auc_49: 0.6909
Epoch 3/10
5409/5409 [==============================] - 2s 315us/step - loss: 0.2750 - accuracy: 0.9079 - f1_score: 0.8908 - auc_49: 0.7689 - val_loss: 0.2019 - val_accuracy: 0.9180 - val_f1_score: 0.9015 - val_auc_49: 0.8240
Epoch 4/10
5409/5409 [==============================] - 2s 305us/step - loss: 0.1860 - accuracy: 0.9388 - f1_score: 0.9327 - auc_49: 0.8571 - val_loss: 0.2868 - val_accuracy: 0.9904 - val_f1_score: 0.9898 - val_auc_49: 0.8822
Epoch 5/10
5409/5409 [==========================

In [361]:
model_1.predict(X[:10])

array([[1.        ],
       [0.9999999 ],
       [0.9830047 ],
       [0.9416699 ],
       [0.18876949],
       [1.        ],
       [0.01592434],
       [0.01592434],
       [0.01592434],
       [0.01592434]], dtype=float32)

In [362]:
Y[:10]

array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

### Model 2 - Fire Image Data and Weather Data

In [345]:
# Create model_2: image data and weather data with functional API

# Define image inputs shape
image_shape = X[0].shape
image_inputs = Input(shape = image_shape)

# Define weather inputs shape


# Add layers for fire image interpretation
output_1 = AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid')(image_inputs)
output_2 = Conv2D(64, kernel_size=(3, 3), activation='sigmoid')(output_1)
output_3 = Conv2D(32, kernel_size=(3, 3), activation='sigmoid')(output_2)
output_4 = MaxPooling2D(pool_size=(2,2), strides=None, padding='valid')(output_3)
output_5 = Dropout(0.2)(output_4)
output_6 = Flatten()(output_5)

# Add layers for weather interpretation


# Combine the layers


# Final dense layer 
predictions = Dense(1, activation='sigmoid')(output_6)

# Define the model
model_2 = Model(inputs=image_inputs, outputs=predictions)

In [346]:
# compile the model
model_2.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy', f1_score, tf.keras.metrics.AUC()]
)

In [347]:
# fit the model
model_2.fit(
    x = [X], 
    y = Y,
    validation_split = test_size, 
    epochs=epoc
)

Train on 5409 samples, validate on 1353 samples
Epoch 1/10
5409/5409 [==============================] - 2s 326us/step - loss: 0.7014 - accuracy: 0.5178 - f1_score: 0.4760 - auc_48: 0.5149 - val_loss: 0.6910 - val_accuracy: 0.5041 - val_f1_score: 0.0000e+00 - val_auc_48: 0.5264
Epoch 2/10
5409/5409 [==============================] - 1s 263us/step - loss: 0.6560 - accuracy: 0.6064 - f1_score: 0.6168 - auc_48: 0.5492 - val_loss: 0.5034 - val_accuracy: 0.8721 - val_f1_score: 0.8446 - val_auc_48: 0.6185
Epoch 3/10
5409/5409 [==============================] - 1s 262us/step - loss: 0.3353 - accuracy: 0.8769 - f1_score: 0.8647 - auc_48: 0.7048 - val_loss: 0.2030 - val_accuracy: 0.9527 - val_f1_score: 0.9488 - val_auc_48: 0.7777
Epoch 4/10
5409/5409 [==============================] - 1s 261us/step - loss: 0.1969 - accuracy: 0.9338 - f1_score: 0.9279 - auc_48: 0.8218 - val_loss: 0.1456 - val_accuracy: 0.9616 - val_f1_score: 0.9586 - val_auc_48: 0.8558
Epoch 5/10
5409/5409 [======================